In [1]:
import pandas as pd

# Load datasets
sales_by_state = pd.read_csv('datasets/electric_vehicle_sales_by_state.csv')
sales_by_makers = pd.read_csv('datasets/electric_vehicle_sales_by_makers.csv')
dim_date = pd.read_csv('datasets/dim_date.csv')

# Convert date columns to datetime
sales_by_state['date'] = pd.to_datetime(sales_by_state['date'], format='%d-%b-%y')
sales_by_makers['date'] = pd.to_datetime(sales_by_makers['date'], format='%d-%b-%y')
dim_date['date'] = pd.to_datetime(dim_date['date'], format='%d-%b-%y')

# Merge the state with date
merged_state = pd.merge(sales_by_state, dim_date, on='date')

# Merge the maker with date
merged_makers = pd.merge(sales_by_makers, dim_date, on='date')

# merged datasets
print(merged_state.head())
print(merged_makers.head())


        date   state vehicle_category  electric_vehicles_sold  \
0 2021-04-01  Sikkim       2-Wheelers                       0   
1 2021-04-01  Sikkim       4-Wheelers                       0   
2 2021-05-01  Sikkim       2-Wheelers                       0   
3 2021-05-01  Sikkim       4-Wheelers                       0   
4 2021-06-01  Sikkim       2-Wheelers                       0   

   total_vehicles_sold  fiscal_year quarter  
0                  398         2022      Q1  
1                  361         2022      Q1  
2                  113         2022      Q1  
3                   98         2022      Q1  
4                  229         2022      Q1  
        date vehicle_category         maker  electric_vehicles_sold  \
0 2021-04-01       2-Wheelers  OLA ELECTRIC                       0   
1 2022-04-01       2-Wheelers      OKAYA EV                       0   
2 2021-05-01       2-Wheelers  OLA ELECTRIC                       0   
3 2021-06-01       2-Wheelers  OLA ELECTRIC      

In [2]:
# 1. List the top 3 and bottom 3 makers for the fiscal years 2023 and 2024 in terms of the number of 2-wheelers sold.

import plotly.express as px

# Filter data for fiscal years 2023 and 2024 and 2-Wheelers
fy_2023_2024_2w = merged_makers[(merged_makers['fiscal_year'].isin([2023, 2024])) & (merged_makers['vehicle_category'] == '2-Wheelers')]

# Sum of electric vehicles sold by maker and fiscal year
makers_sales = fy_2023_2024_2w.groupby(['maker', 'fiscal_year'])['electric_vehicles_sold'].sum().reset_index()

# Top 3 and Bottom 3 Makers for each year
top_bottom_makers = pd.concat([
    makers_sales.groupby('fiscal_year').apply(lambda x: x.nlargest(3, 'electric_vehicles_sold')).reset_index(drop=True),
    makers_sales.groupby('fiscal_year').apply(lambda x: x.nsmallest(3, 'electric_vehicles_sold')).reset_index(drop=True)
])


fig = px.bar(top_bottom_makers, x='maker', y='electric_vehicles_sold', color='fiscal_year',
             title='Top 3 and Bottom 3 Makers for Fiscal Years 2023 and 2024 in Terms of 2-Wheelers Sold',
             labels={'electric_vehicles_sold': 'Electric Vehicles Sold', 'maker': 'Maker'},
             barmode='group')
fig.show()


C:\Users\Asus\AppData\Local\Temp\ipykernel_30632\2255495150.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  makers_sales.groupby('fiscal_year').apply(lambda x: x.nlargest(3, 'electric_vehicles_sold')).reset_index(drop=True),
C:\Users\Asus\AppData\Local\Temp\ipykernel_30632\2255495150.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  makers_sales.groupby('fiscal_year').apply(lambda x: x.nsmallest(3, 

In [3]:
# 2. Identify the top 5 states with the highest penetration rate in 2-wheeler and 4-wheeler EV sales in FY 2024.

# Penetration Rate: This metric represents the percentage of total vehicles that are electric within a specific region or category. It is calculated as:
# 		Penetration Rate =  (Electric Vehicles Sold / Total Vehicles Sold) * 100  
# This indicates the adoption level of electric vehicles.

# Calculate penetration rate 
fy_2024 = merged_state[merged_state['fiscal_year'] == 2024]
fy_2024['penetration_rate'] = (fy_2024['electric_vehicles_sold'] / fy_2024['total_vehicles_sold']) * 100

# Top 5 states by penetration rate for 2-wheelers and 4-wheelers
top_5_states_2w = fy_2024[fy_2024['vehicle_category'] == '2-Wheelers'].groupby('state')['penetration_rate'].mean().nlargest(5).reset_index()
top_5_states_4w = fy_2024[fy_2024['vehicle_category'] == '4-Wheelers'].groupby('state')['penetration_rate'].mean().nlargest(5).reset_index()


fig_2w = px.bar(top_5_states_2w, x='state', y='penetration_rate', 
                title='Top 5 States with the Highest Penetration Rate in 2-Wheeler EV Sales in FY 2024',
                labels={'penetration_rate': 'Penetration Rate (%)', 'state': 'State'})
fig_2w.show()

fig_4w = px.bar(top_5_states_4w, x='state', y='penetration_rate', 
                title='Top 5 States with the Highest Penetration Rate in 4-Wheeler EV Sales in FY 2024',
                labels={'penetration_rate': 'Penetration Rate (%)', 'state': 'State'})
fig_4w.show()


C:\Users\Asus\AppData\Local\Temp\ipykernel_30632\1268383371.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
merged_state

,date,state,vehicle_category,electric_vehicles_sold,total_vehicles_sold,fiscal_year,quarter
0,2021-04-01,Sikkim,2-Wheelers,0,398,2022,Q1
1,2021-04-01,Sikkim,4-Wheelers,0,361,2022,Q1
2,2021-05-01,Sikkim,2-Wheelers,0,113,2022,Q1
3,2021-05-01,Sikkim,4-Wheelers,0,98,2022,Q1
4,2021-06-01,Sikkim,2-Wheelers,0,229,2022,Q1
...,...,...,...,...,...,...,...
2440,2024-03-01,Mizoram,2-Wheelers,58,1932,2024,Q4
2441,2024-03-01,DNH and DD,2-Wheelers,25,780,2024,Q4
2442,2024-03-01,Manipur,2-Wheelers,13,1394,2024,Q4
2443,2024-03-01,Andaman & Nicobar Island,2-Wheelers,2,447,2024,Q4


In [5]:
# 3. List the states with negative penetration (decline) in EV sales from 2022 to 2024?

# Calculate penetration rates for 2022 and 2024
fy_2022_2024 = merged_state[merged_state['fiscal_year'].isin([2022, 2024])]
fy_2022_2024['penetration_rate'] = (fy_2022_2024['electric_vehicles_sold'] / fy_2022_2024['total_vehicles_sold']) * 100

# Penetration rates by state for 2022 and 2024
penetration_2022 = fy_2022_2024[fy_2022_2024['fiscal_year'] == 2022].groupby('state')['penetration_rate'].mean().reset_index()
penetration_2024 = fy_2022_2024[fy_2022_2024['fiscal_year'] == 2024].groupby('state')['penetration_rate'].mean().reset_index()

# Merge and calculate change in penetration rate
penetration_change = pd.merge(penetration_2022, penetration_2024, on='state', suffixes=('_2022', '_2024'))
penetration_change['change'] = penetration_change['penetration_rate_2024'] - penetration_change['penetration_rate_2022']

# Filter states with negative change
negative_penetration_states = penetration_change[penetration_change['change'] < 0]


fig_neg = px.bar(negative_penetration_states, x='state', y='change',
                 title='States with Negative Penetration (Decline) in EV Sales from 2022 to 2024',
                 labels={'change': 'Change in Penetration Rate (%)', 'state': 'State'})
fig_neg.show()


C:\Users\Asus\AppData\Local\Temp\ipykernel_30632\3811289946.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
# 4. What are the quarterly trends based on sales volume for the top 5 EV makers (4-wheelers) from 2022 to 2024?

# Calculate quarterly trends for top 5 makers
fy_2022_2024_4w = merged_makers[merged_makers['vehicle_category'] == '4-Wheelers']
top_5_makers_4w = fy_2022_2024_4w.groupby('maker')['electric_vehicles_sold'].sum().nlargest(5).index
quarterly_trends = fy_2022_2024_4w[fy_2022_2024_4w['maker'].isin(top_5_makers_4w)].groupby(['quarter', 'maker'])['electric_vehicles_sold'].sum().reset_index()


fig_quarterly = px.line(quarterly_trends, x='quarter', y='electric_vehicles_sold', color='maker',
                        title='Quarterly Trends Based on Sales Volume for Top 5 EV Makers (4-Wheelers) from 2022 to 2024',
                        labels={'electric_vehicles_sold': 'Electric Vehicles Sold', 'quarter': 'Quarter'})
fig_quarterly.show()


In [7]:
# 5. How do the EV sales and penetration rates in Delhi compare to Karnataka for 2024?

# Calculate sales and penetration rates for Delhi and Karnataka for 2024
delhi_karnataka_2024 = fy_2024[fy_2024['state'].isin(['Delhi', 'Karnataka'])].groupby(['state', 'vehicle_category'])[['electric_vehicles_sold', 'penetration_rate']].mean().reset_index()


fig_delhi_kar = px.bar(delhi_karnataka_2024, x='state', y='electric_vehicles_sold', color='vehicle_category',
                       title='EV Sales and Penetration Rates in Delhi vs Karnataka for 2024',
                       labels={'electric_vehicles_sold': 'Electric Vehicles Sold', 'state': 'State'},
                       barmode='group')
fig_delhi_kar.show()


In [8]:
# 6. List down the compounded annual growth rate (CAGR) in 4-wheeler units for the top 5 makers from 2022 to 2024.

# Compound Annual Growth Rate (CAGR): CAGR measures the mean annual growth rate over a specified period longer than one year. It is calculated as:
# 		CAGR = [(Ending Value / Beginning Value) ** 1/n] -1

def calculate_cagr(end_value, start_value, periods):
    return (end_value / start_value) ** (1 / periods) - 1



# Filter data for 4-Wheelers for fiscal years 2022 and 2024
fy_2022_2024_4w = merged_makers[(merged_makers['fiscal_year'].isin([2022, 2024])) & (merged_makers['vehicle_category'] == '4-Wheelers')]

# Get top 5 makers based on total electric vehicles sold from 2022 to 2024
top_5_makers = fy_2022_2024_4w.groupby('maker')['electric_vehicles_sold'].sum().nlargest(5).index

cagr_results = []

for maker in top_5_makers:
    maker_data = fy_2022_2024_4w[fy_2022_2024_4w['maker'] == maker].groupby('fiscal_year')['electric_vehicles_sold'].sum()
    start_value = maker_data.loc[2022]
    end_value = maker_data.loc[2024]
    cagr = calculate_cagr(end_value, start_value, 2)  # From 2022 to 2024
    cagr_results.append({'maker': maker, 'CAGR': cagr})

# Convert CAGR results to a DataFrame
cagr_df = pd.DataFrame(cagr_results)
cagr_df['CAGR'] = cagr_df['CAGR'] * 100  # Convert to percentage

#  the results using Plotly
fig_cagr_makers = px.bar(cagr_df, x='maker', y='CAGR',
                         title='CAGR in 4-Wheeler Units for the Top 5 Makers from 2022 to 2024',
                         labels={'CAGR': 'CAGR (%)', 'maker': 'Maker'})
fig_cagr_makers.show()


C:\Users\Asus\AppData\Local\Temp\ipykernel_30632\1679749624.py:7: RuntimeWarning:

divide by zero encountered in scalar divide



In [9]:
# 7. List down the top 10 states that had the highest compounded annual growth rate (CAGR) from 2022 to 2024 in total vehicles sold.

# Group data by state and fiscal year
total_sales_by_state = merged_state.groupby(['state', 'fiscal_year'])['total_vehicles_sold'].sum().unstack()

# Calculate CAGR for each state
cagr_results_state = []

for state in total_sales_by_state.index:
    start_value = total_sales_by_state.loc[state, 2022]
    end_value = total_sales_by_state.loc[state, 2024]
    cagr = calculate_cagr(end_value, start_value, 2)  # From 2022 to 2024
    cagr_results_state.append({'state': state, 'CAGR': cagr})

# Convert to DataFrame and get top 10 states by CAGR
cagr_df_state = pd.DataFrame(cagr_results_state)
cagr_df_state['CAGR'] = cagr_df_state['CAGR'] * 100  # Convert to percentage
top_10_states_cagr = cagr_df_state.nlargest(10, 'CAGR')


fig_cagr_states = px.bar(top_10_states_cagr, x='state', y='CAGR',
                         title='Top 10 States with Highest CAGR in Total Vehicles Sold (2022-2024)',
                         labels={'CAGR': 'CAGR (%)', 'state': 'State'})
fig_cagr_states.show()


In [10]:
merged_state

,date,state,vehicle_category,electric_vehicles_sold,total_vehicles_sold,fiscal_year,quarter
0,2021-04-01,Sikkim,2-Wheelers,0,398,2022,Q1
1,2021-04-01,Sikkim,4-Wheelers,0,361,2022,Q1
2,2021-05-01,Sikkim,2-Wheelers,0,113,2022,Q1
3,2021-05-01,Sikkim,4-Wheelers,0,98,2022,Q1
4,2021-06-01,Sikkim,2-Wheelers,0,229,2022,Q1
...,...,...,...,...,...,...,...
2440,2024-03-01,Mizoram,2-Wheelers,58,1932,2024,Q4
2441,2024-03-01,DNH and DD,2-Wheelers,25,780,2024,Q4
2442,2024-03-01,Manipur,2-Wheelers,13,1394,2024,Q4
2443,2024-03-01,Andaman & Nicobar Island,2-Wheelers,2,447,2024,Q4


In [11]:
# 8. What are the peak and low season months for EV sales based on the data from 2022 to 2024?
merged_state['month'] = merged_state['date'].dt.month

# Group by month and fiscal year to find total EV sales
monthly_sales = merged_state.groupby(['fiscal_year', 'month'])['electric_vehicles_sold'].sum().reset_index()

# Find peak and low season months based on total sales
monthly_sales_avg = monthly_sales.groupby('month')['electric_vehicles_sold'].mean().reset_index()
peak_month = monthly_sales_avg.nlargest(1, 'electric_vehicles_sold')
low_month = monthly_sales_avg.nsmallest(1, 'electric_vehicles_sold')

#  the monthly sales trend
fig_monthly_sales = px.line(monthly_sales, x='month', y='electric_vehicles_sold', color='fiscal_year',
                            title='Monthly EV Sales Trend (2022-2024)',
                            labels={'electric_vehicles_sold': 'Electric Vehicles Sold', 'month': 'Month'})
fig_monthly_sales.show()


In [12]:
# 9. What is the projected number of EV sales (including 2-wheelers and 4-wheelers) for the top 10 states by penetration rate in 2030, based on the compounded annual growth rate (CAGR) from previous years?


# Calculate penetration rate for each state in 2024
merged_state['penetration_rate'] = (merged_state['electric_vehicles_sold'] / merged_state['total_vehicles_sold']) * 100
penetration_2024 = merged_state[merged_state['fiscal_year'] == 2024].groupby('state')['penetration_rate'].mean().reset_index()
top_10_states_penetration = penetration_2024.nlargest(10, 'penetration_rate')

# Project sales for 2030 based on CAGR from 2022 to 2024
projected_sales = []

for state in top_10_states_penetration['state']:
    state_data = total_sales_by_state.loc[state]
    start_value = state_data[2022]
    end_value = state_data[2024]
    cagr = calculate_cagr(end_value, start_value, 2)  # From 2022 to 2024
    projected_value = end_value * ((1 + cagr) ** 6)  # Projecting to 2030
    projected_sales.append({'state': state, 'Projected Sales 2030': projected_value})

projected_sales_df = pd.DataFrame(projected_sales)

#  the projected sales
fig_projected_sales = px.bar(projected_sales_df, x='state', y='Projected Sales 2030',
                             title='Projected Number of EV Sales for Top 10 States by Penetration Rate in 2030',
                             labels={'Projected Sales 2030': 'Projected Sales 2030', 'state': 'State'})
fig_projected_sales.show()


In [13]:
# 10. Estimate the revenue growth rate of 4-wheeler and 2-wheelers EVs in India for 2022 vs 2024 and 2023 vs 2024, assuming an average unit price.

# Assume average unit prices
average_price_2w = 85000  # Example price in INR
average_price_4w = 1500000  # Example price in INR

# Filter data for 2-Wheelers and 4-Wheelers
sales_2w = merged_state[merged_state['vehicle_category'] == '2-Wheelers']
sales_4w = merged_state[merged_state['vehicle_category'] == '4-Wheelers']

# Calculate total revenue for each year
revenue_2w = sales_2w.groupby('fiscal_year')['electric_vehicles_sold'].sum() * average_price_2w
revenue_4w = sales_4w.groupby('fiscal_year')['electric_vehicles_sold'].sum() * average_price_4w

# Calculate revenue growth rates
growth_rate_2w_2022_2024 = (revenue_2w[2024] / revenue_2w[2022]) - 1
growth_rate_4w_2022_2024 = (revenue_4w[2024] / revenue_4w[2022]) - 1

growth_rate_2w_2023_2024 = (revenue_2w[2024] / revenue_2w[2023]) - 1
growth_rate_4w_2023_2024 = (revenue_4w[2024] / revenue_4w[2023]) - 1

# Print the results
print(f"2-Wheeler Revenue Growth Rate (2022-2024): {growth_rate_2w_2022_2024:.2%}")
print(f"4-Wheeler Revenue Growth Rate (2022-2024): {growth_rate_4w_2022_2024:.2%}")

print(f"2-Wheeler Revenue Growth Rate (2023-2024): {growth_rate_2w_2023_2024:.2%}")
print(f"4-Wheeler Revenue Growth Rate (2023-2024): {growth_rate_4w_2023_2024:.2%}")


2-Wheeler Revenue Growth Rate (2022-2024): 269.28%
4-Wheeler Revenue Growth Rate (2022-2024): 367.79%
2-Wheeler Revenue Growth Rate (2023-2024): 28.13%
4-Wheeler Revenue Growth Rate (2023-2024): 83.08%


In [14]:
# Additional analysis which state got highest salen in EVs 

sales=px.scatter(merged_state,
                 x='date',
                 y='electric_vehicles_sold',
                 title="EV Sales by states",
                 labels={'electric_vehicles_sold':'EV Sales','date':'Date'},
                 color='state')
sales.show()

# Secondary Quetions


1. What are the primary reasons for customers choosing 4-wheeler EVs in 2023 and 2024 (cost savings, environmental concerns, government incentives)?

Environmental Concerns

Government Incentives

Rising Fuel Costs

EV technology advancements

Reduced Operating Costs

Urbanization and Traffic Congestion

Global Sustainable Mobility Trends

Charging Infrastructure Development

Advocacy and Awareness Campaigns

Innovative Government Policies

Corporate Fleets and Incentives

Improved Battery Technology

2. How do government incentives and subsidies impact the adoption rates of 2-wheelers and 4-wheelers? Which states in India provided most subsidies?

Government incentives and subsidies have a substantial impact on the adoption rates of both 2-wheelers and 4-wheelers:

Adoption Rates: Incentives lower the upfront costs, making EVs more accessible. For instance, the Faster Adoption and Manufacturing of Hybrid and Electric Vehicles (FAME II) scheme has significantly boosted EV sales
Subsidy Impact by State: States like Delhi, Maharashtra, and Gujarat have provided substantial subsidies and benefits, which have driven higher adoption rates in these regions​ 


3. How does the availability of charging stations infrastructure correlate with the EV sales and penetration rates in the top 5 states?

The top states in India for EV sales, such as Maharashtra, Karnataka, Tamil Nadu, Gujarat, and Delhi, have invested significantly in developing their charging infrastructure, correlating with higher EV adoption rates​

4. Who should be the brand ambassador if AtliQ Motors launches their EV/Hybrid vehicles in India and why?

Virat Kohli is an excellent choice for the brand ambassador due to his immense popularity, trustworthiness, and strong appeal among diverse demographics in India. His commitment to fitness and a sustainable lifestyle aligns well with the eco-friendly image of EVs. Additionally, his influence extends beyond cricket fans, making him a powerful spokesperson for promoting new technologies and green initiatives.

5. Which state of India is ideal to start the manufacturing unit? (Based on subsidies provided, ease of doing business, stability in governance etc.)

Gujarat stands out as an ideal state to start the manufacturing unit for several reasons:

Subsidies and Incentives: Gujarat offers attractive subsidies for setting up EV manufacturing units.

Ease of Doing Business: The state has a business-friendly environment with streamlined regulations.

Stable Governance: Consistent policies and political stability make long-term investments secure.

Infrastructure: Well-developed industrial infrastructure supports efficient manufacturing and logistics.


6. Your top 3 recommendations for AtliQ Motors.

Invest in Charging Infrastructure: Collaborate with local governments and private partners to develop a robust network of charging stations to alleviate range anxiety and boost consumer confidence.

Focus on Affordability: Offer competitive pricing and financing options to make EVs accessible to a broader customer base, leveraging government incentives to reduce costs.

Marketing and Awareness Campaigns: Launch comprehensive marketing campaigns highlighting the benefits of EVs, such as cost savings, environmental impact, and government incentives, to educate and attract potential buyers.